In [18]:
import fusion_gym_environment
from fusion_gym import fusion_env_wrapper
from model.Operation import Operation
from model.ExtrudeAction import ExtrudeAction

import importlib
import os 
import sys
import uuid
from pathlib import Path

from tf_agents.environments import utils

import numpy as np

importlib.reload(fusion_gym_environment)

##############################
##### Fusion gym imports #####
##############################

# Add the client folder to sys.path
CLIENT_DIR = os.path.join("..", "tools", "fusion360gym", "client")
if CLIENT_DIR not in sys.path:
    sys.path.append(CLIENT_DIR)

from gym_env import GymEnv

In [19]:
face1 = "850e641c-e309-11ea-ad98-54bf646e7e1f"
face2 = "850eb246-e309-11ea-a0ba-54bf646e7e1f"

action = fusion_gym_environment.ExtrudeAction(
    uuid.UUID(face1),
    uuid.UUID(face2),
    Operation.CutFeatureOperation
)

encoded = action.encode()
decoded = fusion_gym_environment.ExtrudeAction.decode(encoded)
assert decoded == action

In [23]:
# Setup Gym client
gym_env = fusion_env_wrapper.FusionEnvWrapper(host="127.0.0.1", port=8080, launch_gym=False)

In [24]:
environment = fusion_gym_environment.FusionGymEnvironment(gym_env, 5)
utils.validate_py_environment(environment, episodes=5)

Exception: [revert_to_target] 500: Failed processing revert_to_target command due to Target not set

In [5]:
import json
import os
from pathlib import Path
from typing import List

def get_files(input_dir: str, split_file: str = None) -> List[str]:
    """ Get list of .smt files from the target directory 
    input_dir -- the directory input files
    split -- the path of the test file
    """
    input_dir = Path(os.path.expanduser(input_dir))
    
    if not input_dir.exists():
        print("Input file/folder does not exist")
        return []
        
    test_files = None
    if split_file is not None:
        split_file = Path(os.path.expanduser(split_file))
        if not split_file.exists():
            print("Split file does not exists")
        else:
            with open(split_file, encoding="utf8") as f:
                json_data = json.load(f)
                if "test" not in json_data:
                    print("Split file does not have a test set")
                else:
                    test_files = set()
                    for test_file in json_data["test"]:
                        test_files.add(f"{test_file}.smt")

    files = []
    if input_dir.is_dir():
        smt_files = [f for f in input_dir.glob("**/*.smt")]
        if len(smt_files) == 0:
            print("No .smt files found")
            exit()
        else:
            # We store an index with the file for debug output
            for smt_file in smt_files:
                if test_files is None:
                    # If we don't have a split
                    # use all files
                    files.append(smt_file)
                else:
                    # If we have a split
                    # use only the test files
                    if smt_file.name in test_files:
                        files.append(smt_file)
    else:
        files.append(input)
    return files

In [25]:
INPUT_DIR = "~/Downloads/r1.0.0/reconstruction/"
input_files = get_files(INPUT_DIR)

In [26]:
def parse_sequence(sequence_file) -> list:
    with open(sequence_file, 'r') as f:
        contents = json.loads(f.read())
        return contents["sequence"]
    return []

In [27]:
file = Path(f"/Users/erikbeerepoot/Downloads/r1.0.0/reconstruction/20241_6bced5ac_0000_0001.smt")
#file = Path(f"/Users/erikbeerepoot/Downloads/regraph_05/20241_6bced5ac_0000_0000.json")
sequence_file = Path("/Users/erikbeerepoot/Downloads/regraph_05/20241_6bced5ac_0000_sequence.json")
gym_env.set_target(file)

({'directed': False,
  'multigraph': False,
  'graph': {},
  'nodes': [{'id': '0',
    'surface_type': 'PlaneSurfaceType',
    'points': [11.545454545454545,
     5.08,
     0.23090909090909092,
     11.545454545454545,
     5.08,
     0.46181818181818185,
     11.545454545454545,
     5.08,
     0.6927272727272727,
     11.545454545454545,
     5.08,
     0.9236363636363637,
     11.545454545454545,
     5.08,
     1.1545454545454545,
     11.545454545454545,
     5.08,
     1.3854545454545455,
     11.545454545454545,
     5.08,
     1.6163636363636364,
     11.545454545454545,
     5.08,
     1.8472727272727274,
     11.545454545454545,
     5.08,
     2.078181818181818,
     11.545454545454545,
     5.08,
     2.309090909090909,
     10.39090909090909,
     5.08,
     0.23090909090909092,
     10.39090909090909,
     5.08,
     0.46181818181818185,
     10.39090909090909,
     5.08,
     0.6927272727272727,
     10.39090909090909,
     5.08,
     0.9236363636363637,
     10.3909090

In [17]:
import time

face1 = "856a3bde-e309-11ea-bdb8-54bf646e7e1f"
face2 = "856de574-e309-11ea-9295-54bf646e7e1f"

for step in parse_sequence(sequence_file):
    action = fusion_gym_environment.ExtrudeAction(
        uuid.UUID(step["start_face"]),
        uuid.UUID(step["end_face"]),
        Operation.from_string(step["operation"])
    )
    environment.step(action.encode())
    time.sleep(2)



step: State(step=3, finished=False)
ExtrudeAction(start_face=UUID('94e09a3a-f2df-11ea-95f4-066234a457d1'), end_face=UUID('94e0b2ba-f2df-11ea-8c36-066234a457d1'), operation=<Operation.NewBodyFeatureOperation: 4>)
args: 94e09a3a-f2df-11ea-95f4-066234a457d1, 94e0b2ba-f2df-11ea-8c36-066234a457d1, NewBodyFeatureOperation


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x17ca59f60>: Failed to establish a new connection: [Errno 61] Connection refused'))